In [29]:
import requests
from bs4 import BeautifulSoup
from user_agent import generate_user_agent


url = "https://www.amazon.com.mx/s"  # URL de búsqueda en Amazon México

# Parámetros para la búsqueda
params = {
    'k': 'freidora'  # Término de búsqueda, en este caso "freidora"
}

# Encabezados personalizados
custom_headers = {
    'user-agent': generate_user_agent(),  # Generar un User-Agent aleatorio
    'accept-language': 'en-GB,en;q=0.9', # mejorar los headers 
}

# Lista para almacenar los enlaces de los productos
product_links_list = []
num_products = 500  # Número de productos a recolectar
current_url = url  # URL inicial para comenzar la búsqueda

while len(product_links_list) < num_products:
    response = requests.get(current_url, params=params, headers=custom_headers)
    
    # Comprobamos que la solicitud fue exitosa
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        product_links = soup.find_all("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")

        for product in product_links:
            if len(product_links_list) >= num_products:
                break

            link = "https://www.amazon.com.mx" + product['href']
            product_links_list.append(link)
        
        next_page = soup.find("a", class_="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator")
        
        if next_page and 'href' in next_page.attrs:
            current_url = "https://www.amazon.com.mx" + next_page['href']
        else:
            print("yap quedo o no hay massss ")
            break
    else:
        print("No se pudo acceder a la página. Status code:", response.status_code)
        break

# Imprimir los enlaces de productos recolectados
print(f"Total de enlaces recolectados: {len(product_links_list)}")
for link in product_links_list:
    print(link)


yap quedo o no hay massss 
Total de enlaces recolectados: 390
https://www.amazon.com.mx/sspa/click?ie=UTF8&spc=MTo3ODU2NDA3NDI0MzIzMjAzOjE3MzA3ODYxMzA6c3BfYXRmOjMwMDAyODY4MTE1NjkwMjo6MDo6&url=%2FChefman-Freidora-Digital-Deshidratador-Rostizador%2Fdp%2FB08DL8WH9V%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.7Be5egMASqYmuPqJkA5N5nYFylERg5VlmY4sa3pYXShoNUwR6wMjbodKuXUPVVIa-WVQYQ-HgxS9BMUAHXEk5nFtlg7J5NwxHLKk3hCNbZVsvGA1QEXAU0s6bBASX8jNc8btn7VD4YqJG8NccJ4o8QxMaPR37CVMikMdkQXmrexOpmS5vq4g9BCEnm8rZDwy0Ysd6Y3-ispfdVDwaxhrWTk4hFmRepSAWkshiHpLzXG2gGuM8sRcR0J_gNptxT2HBJQVbSuDWuiYXKoa_69Z9-6POGZql0Z5FqC1PGsUMbI._lMAkuuUAfsTnYL0rReAz0GB9vB7zMirLWvwxNJ0_jY%26dib_tag%3Dse%26keywords%3Dfreidora%26qid%3D1730786130%26sr%3D8-1-spons%26ufe%3Dapp_do%253Aamzn1.fos.de93fa6a-174c-4df7-be7c-5bc8e9c5a71b%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
https://www.amazon.com.mx/Hamilton-Beach-35210-Profesional-Inoxidable/dp/B08YMWQF2X/ref=sr_1_2?dib=eyJ2IjoiMSJ9.7Be5egMASqYmuPqJkA5N5nYFylERg5VlmY4sa3pYXShoNUwR6wMjbodK

In [30]:
def getReviewsAmazon(link):
    import requests
    from bs4 import BeautifulSoup
    from user_agent import generate_user_agent
    import re

    # Encabezados personalizados
    custom_headers = {
        'user-agent': generate_user_agent(),  # Generar un User-Agent aleatorio
        'accept-language': 'en-US,en;q=0.9',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'referer': 'https://www.amazon.com.mx/',
        'upgrade-insecure-requests': '1'
    }

    # Hacemos la solicitud GET a la página del producto
    response = requests.get(link, headers=custom_headers)
    
    # Comprobamos que la solicitud fue exitosa
    if response.status_code == 200:
        # Crear el objeto BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Buscar el enlace para ver todas las opiniones
        reviews_link_element = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})
        
        # Verificar si se encontró el enlace
        if reviews_link_element and 'href' in reviews_link_element.attrs:
            # Construir la URL completa de la sección de opiniones
            reviews_link = "https://www.amazon.com.mx" + reviews_link_element['href']
            print("Enlace a todas las opiniones:", reviews_link)
            
            # Hacer una solicitud GET al enlace de opiniones
            reviews_response = requests.get(reviews_link, headers=custom_headers)
            
            # Comprobar que la solicitud fue exitosa
            if reviews_response.status_code == 200:
                reviews_soup = BeautifulSoup(reviews_response.text, 'html.parser')
                # Aquí procesamos la página de opiniones
                print("Página de opiniones accedida correctamente.")
                
                # Extraer el número de reseñas con opiniones
                review_info_element = reviews_soup.find("div", {"data-hook": "cr-filter-info-review-rating-count"})
                if review_info_element:
                    review_info_text = review_info_element.get_text(strip=True)
                    match = re.search(r"(\d{1,3}(?:,\d{3})*) con opiniones", review_info_text)
                    
                    if match:
                        reviews_with_opinions = int(match.group(1).replace(",", ""))
                        print(f"Número de reseñas con opiniones: {reviews_with_opinions}")

                        # Buscar todos los elementos de texto de reseñas
                        review_texts = reviews_soup.find_all("span", class_="a-size-base review-text review-text-content")
                        
                        # Almacenar los textos de reseña en una lista
                        reviews_content = []
                        for review in review_texts:
                            # Extraer el texto de la reseña
                            review_text = review.get_text(strip=True)
                            reviews_content.append(review_text)
                        
                        # Imprimir los textos de reseña encontrados
                        print("Textos de reseñas encontrados:")
                        for text in reviews_content:
                            print(text)
                        
                        return reviews_with_opinions, reviews_content
                    else:
                        print("No se encontró el número de reseñas con opiniones.")
                        return None
                else:
                    print("No se encontró el elemento con data-hook='cr-filter-info-review-rating-count'.")
                    return None
            else:
                print("No se pudo acceder a la página de opiniones. Status code:", reviews_response.status_code)
                return None
        else:
            print("No se encontró el enlace a todas las opiniones.")
            return None
    else:
        print("No se pudo acceder a la página del producto. Status code:", response.status_code)
        return None

getReviewsAmazon(product_links_list[0])



Enlace a todas las opiniones: https://www.amazon.com.mx/Chefman-Freidora-Digital-Deshidratador-Rostizador/product-reviews/B08DL8WH9V/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
Página de opiniones accedida correctamente.
Número de reseñas con opiniones: 3859
Textos de reseñas encontrados:
Excelente, es super práctico y conveniente, en 5 minutos pueden salir varias carnes a la vez, 10 de 10.
Funciona muy bien y no e tenido ningún problema
Es muy práctica solo se debe voltear los alimentos que están en la parte de abajo o quedan crudos.
Muy bonita, es pequeña lo que hace que el acomodo sea sencillo.La caja llego algo maltratada, pero al parecer el producto esta intacto.No llego el cesto giratorio y fue una de las caracteristicas por las que me emociono comprarla. Sido mi primera freidoraAlguien me puede guiar como contactar al vendedor para solicitar la pieza de cesta giratoria que falto por favor
Cambié mi previa freidora de 3.7 L por esta de 10 (en oferta me costó prác

(3859,
 ['Excelente, es super práctico y conveniente, en 5 minutos pueden salir varias carnes a la vez, 10 de 10.',
  'Funciona muy bien y no e tenido ningún problema',
  'Es muy práctica solo se debe voltear los alimentos que están en la parte de abajo o quedan crudos.',
  'Muy bonita, es pequeña lo que hace que el acomodo sea sencillo.La caja llego algo maltratada, pero al parecer el producto esta intacto.No llego el cesto giratorio y fue una de las caracteristicas por las que me emociono comprarla. Sido mi primera freidoraAlguien me puede guiar como contactar al vendedor para solicitar la pieza de cesta giratoria que falto por favor',
  'Cambié mi previa freidora de 3.7 L por esta de 10 (en oferta me costó prácticamente lo mismo hace un año y también en oferta la anterior.  Esto porque aprendí las ventajas y lo que podía hacer en ella y el volumen de la primera ya no era suficiente.La sorpresa es que para ser de más volúmen, no ocupa tanto espacio como imaginaba y también aprecié mu

In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from user_agent import generate_user_agent

# Set up options for headers
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode if desired
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# Set user-agent to mimic a real browser
headers = {
    "User-Agent": generate_user_agent(),
    'accept-language': 'en-US,en;q=0.9',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'referer': 'https://www.amazon.com.mx/Hamilton-Beach-35034-Freidora/dp/B00M39ML76/ref=sr_1_1?dib=eyJ2IjoiMSJ9.AHRpCASmeBcoCU1sKgIWPCS5J0_YJ9R3ivEWYegI6o5RefUvLVuKzj4JDuwdKLwWDHVVEW62qt_cCuiPqtc1X0Eti0bnyzsR33U88J5sSw7Zx10-xT0VIxNitysXvwxHsOz1Vwp6Ep-QwzLZ6foW0JdQmwQ0tdldYgtLHQ3jVdbgJrYydBDvklxy75rkXzZb_UoWrwwbOCy6irdWg6DsvlDXIpmDnjpH_Mjx8c7t-ESX99zhwnhG8j1rGzq6whgCcxHGvpcIw8I-Jrr5lxm2ByIvPdG2lIupz92ucXUh_VM.JAvtn_4JSlSKzJAgkBLxBtKoXuin23i7kuAX5j5-rYM&dib_tag=se&keywords=freidora&qid=1730093152&sr=8-1&ufe=app_do%3Aamzn1.fos.3ad1a8a7-7417-4467-b505-11834e377abf&th=1',
    'upgrade-insecure-requests': '1'
}

for key, value in headers.items():
    chrome_options.add_argument(f"{key}={value}")

# Start WebDriver
driver = webdriver.Chrome(options=chrome_options)

try:
    # Open the URL
    driver.get(product_links_list[0])
    print(driver.title)
    # Loop to continue clicking the "next" button until it is no longer available
    while True:
        # Extract and print reviews on the current page
        reviews = driver.find_elements(By.CLASS_NAME, "review-text-content")
        for review in reviews:
            print(review.text)
        # Attempt to find and click the "next" button
        try:
            next_button = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[12]/span/div/ul/li[2]/a")
            print(driver.title)
            if next_button.is_enabled():
                next_button.click()
                print("Clicked next button.")
                time.sleep(2)  # Pause briefly to allow the next page to load
            else:
                print("Next button found but not clickable.")
                break
        except (NoSuchElementException, ElementClickInterceptedException):
            print("Next button not found or not clickable.")
            break

finally:
    # Close WebDriver
    driver.quit()


Chefman - Freidora digital multifuncional + asador, deshidratador, horno de convección, 17 preselecciones en pantalla táctil para freír, asar, deshidratar y hornear accesorios incluidos, L 10L, negro : Amazon.com.mx: Hogar y Cocina
Excelente, es super práctico y conveniente, en 5 minutos pueden salir varias carnes a la vez, 10 de 10.
Funciona muy bien y no e tenido ningún problema
Es muy práctica solo se debe voltear los alimentos que están en la parte de abajo o quedan crudos.
Muy bonita, es pequeña lo que hace que el acomodo sea sencillo.
La caja llego algo maltratada, pero al parecer el producto esta intacto.
No llego el cesto giratorio y fue una de las caracteristicas por las que me emociono comprarla. Sido mi primera freidora
Alguien me puede guiar como contactar al vendedor para solicitar la pieza de cesta giratoria que falto por favor
Cambié mi previa freidora de 3.7 L por esta de 10 (en oferta me costó prácticamente lo mismo hace un año y también en oferta la anterior. Esto por

In [32]:
print(len(reviews))

13
